# CAFA 5 Competition : Protein Function Prediction

# Problem Framing

This Kaggle competition aims to predict the function of proteins using their amino-acid sequences and additional data. Understanding protein function is crucial for comprehending cellular processes and developing new treatments for diseases. With the abundance of genomic sequence data available, assigning accurate biological functions to proteins becomes challenging due to their multifunctionality and interactions with various partners. This competition, hosted by the Function Community of Special Interest (Function-COSI), brings together computational biologists, experimental biologists, and biocurators to improve protein function prediction through data science and machine learning approaches. The goal is to contribute to advancements in medicine, agriculture, and overall human and animal health.

![image-intro](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4a/Computational_solvent_mapping_of_AMA1_using_FTMAP.TIF/lossy-page1-500px-Computational_solvent_mapping_of_AMA1_using_FTMAP.TIF.jpg)

# What to submit ?

This competition evaluates participants' predictions of Gene Ontology (GO) terms for protein sequences. The evaluation is performed on a test set of proteins that initially have no assigned functions but may accumulate experimental annotations after the submission deadline. The test set is divided into three subontologies: Molecular Function (MF), Biological Process (BP), and Cellular Component (CC). Participants are scored separately for each subontology. The final performance measure is the arithmetic mean of the maximum F-measures calculated on the three subontologies. Weighted precision and recall are used, taking into account the hierarchical structure of the GO. The evaluation code is publicly available. The leaderboard displays performance on a subset of proteins not included in the final test set, so generalization performance is crucial. Submission files should contain protein-target pairs with corresponding GO terms and estimated probabilities, within a specific score range. The predictions are propagated to parent terms if not explicitly listed. There is a limit on the number of terms associated with each protein. If a protein is not listed in the submission file, it is assumed that all predictions for that protein are zero.

In [1]:
# import pandas as pd
# sub = pd.read_csv("/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv", sep= "\t", header = None)
# sub.columns = ["The Protein ID", "The Gene Ontology term (GO) ID", "Predicted link probability that GO appear in Protein"]
# sub.head(5)

The evaluation metric used in this competition is the weighted F-measure, which combines precision and recall, taking into account the hierarchical structure of the Gene Ontology (GO). The formula for calculating the weighted F-measure is as follows:

$Weighted \ F-measure = \frac{(1 + β^2) * (weighted \ precision * weighted recall)}{((β^2 * weighted \ precision) + weighted \ recall)}$

where:
- β is a parameter that controls the trade-off between precision and recall. In this competition, β is set to 1, resulting in an equal weighting of precision and recall.
- Weighted precision is the precision score, considering the weights of the predicted terms. It is calculated as the sum of the products of the predicted term's weight and its true positive count, divided by the sum of the weights of all predicted terms.
- Weighted recall is the recall score, considering the weights of the true positive terms. It is calculated as the sum of the products of the true positive term's weight and its count, divided by the sum of the weights of all true positive terms.

Note: The specific formulas for weighted precision and weighted recall are provided in the competition materials and utilize additional information such as term weights and true positive counts.

# General Baseline in this Notebook

- **1 - Collect Embedding vectors from pre-trained protein function prediction models (T5, ProtBERT or EMS2) :**

Sources for embeddings vectors : 
- *T5* : https://www.kaggle.com/datasets/sergeifironov/t5embeds

- *ProtBERT* : https://www.kaggle.com/datasets/henriupton/protbert-embeddings-for-cafa5

- *EMS2* : https://www.kaggle.com/datasets/viktorfairuschin/cafa-5-ems-2-embeddings-numpy

- **2 - Generate labels from train_terms file** : by considering the top K most common GO terms in all Proteins set, generate for each protein a sparse vector of length K to indicate the true probabilities that each of the K GO terms are in the Protein (0 or 1). Here we retain K = 600

- **3 - Create Pytorch Dataset class that can handle Protein ID and embeddings**.

- **4 - Create Pytorch Model class for prediction** : can be any architecture of Multilabel classification model that can turn embeddings of shape (E,) to probabilities of shape (K,). Here we explore **MultiLayerPerceptron** and **ConvNN1d** Networks.

- **5 - Make Cross Validation w.r.t the F-1 measure and do Hyperparameter tuning thanks to Weights and Biases package (Wandb)**

![baseline-image](https://www.researchgate.net/publication/334642149/figure/fig1/AS:783995214249986@1563930433525/Flow-chart-of-STRING2GO-based-protein-function-prediction-method.png)

## **Flow-chart of STRING2GO-based protein function prediction method**

# So now lets get started !

# 1. Imports / Config

In [2]:
MAIN_DIR = "/kaggle/input/cafa-5-protein-function-prediction"

# UTILITARIES
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import gc
# import tensorflow as tf
plt.style.use('ggplot')

# TORCH MODULES FOR METRICS COMPUTATION :
import torch
from torch.utils.data import Dataset
from torch import nn
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MultilabelF1Score
from torchmetrics.classification import MultilabelAccuracy

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

# WANDB FOR LIGHTNING :
import wandb

# FILES VISUALIZATION
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo
/kaggle/input/train-tragets-top-500/train_targets_BPO_top_500.pkl
/kaggle/input/train-tragets-top-500/train_targets_MFO_top_500.pkl
/kaggle/input/train-tragets-top-500/train_targets_CCO_top_500.pkl
/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy
/kaggle/input/protbert-embeddings-for-cafa5/train_embeddings.npy
/kaggle/input/protbert-embeddings-for-cafa5/test_ids.npy
/kaggle/input/protbert-embeddi

In [3]:
class config:
    train_sequences_path = MAIN_DIR  + "/Train/train_sequences.fasta"
    train_labels_path = MAIN_DIR + "/Train/train_terms.tsv"
    test_sequences_path = MAIN_DIR + "/Test (Targets)/testsuperset.fasta"
    
    num_labels = 500
    n_epochs = 10
    batch_size = 128
    lr = 1e-3
    early_stop = 7
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(config.device)

cuda


# 2. Baseline

- Use ProtBERT/T5 embedding vectors for each Sequence ID and its associated Protein Sequence. 

- Define a Pytorch Dataset to load all ids/sequences of the train+test sets and their respective ProtBert embeddings

- Define Pytorch Model architecture in order to use these embeddings to proceed classification task : to each ID we associate a probability to be associated to each GO term **For this part, just consider the top_n most common GO terms as labels**

- Return desired probabilities for test set

# 3. Collect ProtBERT Embedding Vectors

### Here is a Dataset Card I have created if you want to understand better what are pre-trained model embedding vectors, and if you want to download it directly from it : https://www.kaggle.com/datasets/henriupton/protbert-embeddings-for-cafa5

##### SCRIPT FOR VECTOR EMBEDDINGS COLLECTING #####
#### RUN THIS ONLY ONE TIME AND SAVE IT IN LOCAL ####

```python

print("Load ProtBERT Model...")
# PROT BERT LOADING :
from transformers import BertModel, BertTokenizer
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )
model = BertModel.from_pretrained("Rostlab/prot_bert").to(config.device)

def get_bert_embedding(
    sequence : str,
    len_seq_limit : int
):
    '''
    Function to collect last hidden state embedding vector from pre-trained ProtBERT Model
    
    INPUTS:
    - sequence (str) : protein sequence (ex : AAABBB) from fasta file
    - len_seq_limit (int) : maximum sequence lenght (i.e nb of letters) for truncation
    
    OUTPUTS:
    - output_hidden : last hidden state embedding vector for input sequence of length 1024
    '''
    sequence_w_spaces = ' '.join(list(sequence))
    encoded_input = tokenizer(
        sequence_w_spaces,
        truncation=True,
        max_length=len_seq_limit,
        padding='max_length',
        return_tensors='pt').to(config.device)
    output = model(**encoded_input)
    output_hidden = output['last_hidden_state'][:,0][0].detach().cpu().numpy()
    assert len(output_hidden)==1024
    return output_hidden

### COLLECTING FOR TRAIN SAMPLES :
print("Loading train set ProtBERT Embeddings...")
fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_train)))
fasta_train = SeqIO.parse(config.train_sequences_path, "fasta")
ids_list = []
embed_vects_list = []
t0 = time.time()
checkpoint = 0
for item in tqdm(fasta_train):
    ids_list.append(item.id)
    embed_vects_list.append(
        get_bert_embedding(sequence = item.seq, len_seq_limit = 1200))
    checkpoint+=1
    if checkpoint>=100:
        df_res = pd.DataFrame(data={"id" : ids_list, "embed_vect" : embed_vects_list})
        np.save('/kaggle/working/train_ids.npy',np.array(ids_list))
        np.save('/kaggle/working/train_embeddings.npy',np.array(embed_vects_list))
        checkpoint=0
        
np.save('/kaggle/working/train_ids.npy',np.array(ids_list))
np.save('/kaggle/working/train_embeddings.npy',np.array(embed_vects_list))
print('Total Elapsed Time:',time.time()-t0)

### COLLECTING FOR TEST SAMPLES :
print("Loading test set ProtBERT Embeddings...")
fasta_test = SeqIO.parse(config.test_sequences_path, "fasta")
print("Total Nb of Elements : ", len(list(fasta_test)))
fasta_test = SeqIO.parse(config.test_sequences_path, "fasta")
ids_list = []
embed_vects_list = []
t0 = time.time()
checkpoint=0
for item in tqdm(fasta_test):
    ids_list.append(item.id)
    embed_vects_list.append(
        get_bert_embedding(sequence = item.seq, len_seq_limit = 1200))
    checkpoint+=1
    if checkpoint>=100:
        np.save('/kaggle/working/test_ids.npy',np.array(ids_list))
        np.save('/kaggle/working/test_embeddings.npy',np.array(embed_vects_list))
        checkpoint=0
        
np.save('/kaggle/working/test_ids.npy',np.array(ids_list))
np.save('/kaggle/working/test_embeddings.npy',np.array(embed_vects_list))
print('Total Elasped Time:',time.time()-t0)
```

# 4. Collect labels vectors for train/test

In [5]:
class ProteinSequenceDataset(Dataset):
    
    def __init__(self, datatype, embeddings_source, aspect=None):
        super(ProteinSequenceDataset).__init__()
        self.datatype = datatype
        
        if embeddings_source in ["ProtBERT", "EMS2", "Ensemble2"]:
            embeds = np.load("/kaggle/input/"+embeds_map[embeddings_source]+"/"+datatype+"_embeddings.npy")
            ids = np.load("/kaggle/input/"+embeds_map[embeddings_source]+"/"+datatype+"_ids.npy")
        
        if embeddings_source in ["T5", "Ensemble1"]:
            embeds = np.load("/kaggle/input/"+embeds_map[embeddings_source]+"/"+datatype+"_embeds.npy")
            ids = np.load("/kaggle/input/"+embeds_map[embeddings_source]+"/"+datatype+"_ids.npy")
            
        embeds_list = []
        for l in range(embeds.shape[0]):
            embeds_list.append(embeds[l,:])
        self.df = pd.DataFrame(data={"EntryID": ids, "embed" : embeds_list})
        
        if datatype=="train":
            if not aspect:
                df_labels = pd.read_pickle(
                    "/kaggle/input/train-tragets-top-500" + \
                    "/train_targets_top"+str(config.num_labels)+".pkl")
            else:
                df_labels = pd.read_pickle(
                    f"/kaggle/input/train-tragets-top-500/train_targets_{aspect}_top_{config.num_labels}.pkl")
            self.df = self.df.merge(df_labels, on="EntryID",how="right")
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        embed = torch.tensor(self.df.iloc[index]["embed"] , dtype = torch.float32)
        if self.datatype=="train":
            targets = torch.tensor(self.df.iloc[index]["labels_vect"], dtype = torch.float32)
            return embed, targets
        if self.datatype=="test":
            id = self.df.iloc[index]["EntryID"]
            return embed, id


##### SCRIPT FOR LABELS (TARGETS) VECTORS COLLECTING #####
#### RUN THIS ONLY ONE TIME AND SAVE IT IN LOCAL ####

```python


print("GENERATE TARGETS FOR ENTRY IDS ("+str(config.num_labels)+" MOST COMMON GO TERMS)")
ids = np.load("/kaggle/input/protbert-embeddings-for-cafa5/train_ids.npy")
labels = pd.read_csv(config.train_labels_path, sep = "\t")

top_terms = labels.groupby("term")["EntryID"].count().sort_values(ascending=False)
labels_names = top_terms[:config.num_labels].index.values
train_labels_sub = labels[(labels.term.isin(labels_names)) & (labels.EntryID.isin(ids))]
id_labels = train_labels_sub.groupby('EntryID')['term'].apply(list).to_dict()

go_terms_map = {label: i for i, label in enumerate(labels_names)}
labels_matrix = np.empty((len(ids), len(labels_names)))

for index, id in tqdm(enumerate(ids),total = len(ids), position=0, leave=True):
    id_gos_list = id_labels[id]
    temp = [go_terms_map[go] for go in labels_names if go in id_gos_list]
    labels_matrix[index, temp] = 1


# 将labels_matrix转换为一个向量列表
vectors = [row.ravel() for row in labels_matrix]

# 创建一个新的DataFrame，其中包含向量和EntryID
df = pd.DataFrame({'EntryID': ids, 'labels_vect': vectors})

# 将DataFrame保存为pkl文件
df.to_pickle("train_targets_top500.pkl")

assert 1==0

```

# 5. Pytorch Dataset Architecture

In [6]:
# Directories for the different embedding vectors : 
embeds_map = {
    "T5" : "t5embeds",
    "ProtBERT" : "protbert-embeddings-for-cafa5",
    "EMS2" : "cafa-5-ems-2-embeddings-numpy",
    "Ensemble1": "ensemble-embedding-for-cfa",
    "Ensemble2": "ensemble-embedding-for-cfa",

}

# Length of the different embedding vectors :
embeds_dim = {
    "T5" : 1024,
    "ProtBERT" : 1024,
    "EMS2" : 1280,
    "Ensemble1": 3328,
    "Ensemble2": 2304,

}

# 6. Pytorch Models Architectures

In [7]:
class MultiLayerPerceptron(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MultiLayerPerceptron, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 1012),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(1012, 712),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.2),
            torch.nn.Linear(712, 700),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.15),
            torch.nn.Linear(700, num_classes),
        )

    def forward(self, x):
        x = self.model(x)
        return x


In [8]:
# class MultiLayerPerceptron(torch.nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(MultiLayerPerceptron, self).__init__()
#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_dim, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(1024, 2048),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
# #             torch.nn.Linear(2048, 2048),
# #             torch.nn.ReLU(),
# #             torch.nn.Dropout(0.2),
#             torch.nn.Linear(2048, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
# #             torch.nn.Linear(1024, 1024),
# #             torch.nn.ReLU(),
# #             torch.nn.Dropout(0.2),
#             torch.nn.Linear(1024, 512),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(512, num_classes),
#         )

#     def forward(self, x):
#         x = self.model(x)
#         return x


In [9]:
# class MultiLayerPerceptron(torch.nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(MultiLayerPerceptron, self).__init__()
#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_dim, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(1024, 2048),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(2048, 2048),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(2048, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(1024, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Dropout(0.2),
#             torch.nn.Linear(1024, num_classes),
#         )

#     def forward(self, x):
#         x = self.model(x)
#         return x


In [10]:
class CNN1D(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(CNN1D, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3, dilation=1, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(in_channels=3, out_channels=8, kernel_size=3, dilation=1, padding=1, stride=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        self.fc = nn.Sequential(
            nn.Linear(in_features=int(8*input_dim/4), out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=num_classes)
        )

    def forward(self, x):
        x = x.reshape(x.shape[0], 1, x.shape[1])
        x = self.conv(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [11]:
class BiLSTM(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim=128, num_layers=2):
        super(BiLSTM, self).__init__()
        self.input_dim = input_dim
        self.lstm = nn.LSTM(input_size=self.input_dim, hidden_size=hidden_dim, num_layers=num_layers, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(in_features=hidden_dim*4, out_features=num_classes)
        )

    def forward(self, x):
        x = x.view(x.shape[0], -1, self.input_dim).transpose(0, 1)
        output, (hn, cn) = self.lstm(x)
        hn = hn.view(hn.shape[1], -1)
        x = self.fc(hn)
        return x


# 7. Train the Model

In [12]:
def train_model(embeddings_source, model_type="linear", train_size=0.9, model_aspect=None):
    if not model_aspect:
        train_dataset = ProteinSequenceDataset(datatype="train", embeddings_source = embeddings_source)
        model_aspect = ''
    else:
        train_dataset = ProteinSequenceDataset(datatype="train", embeddings_source = embeddings_source, aspect = model_aspect)
    
    train_set, val_set = random_split(train_dataset, lengths = [int(len(train_dataset)*train_size), len(train_dataset)-int(len(train_dataset)*train_size)])
    train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=config.batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=config.batch_size, shuffle=True)

    if model_type == "linear":
        model = MultiLayerPerceptron(input_dim=embeds_dim[embeddings_source], num_classes=config.num_labels).to(config.device)
    if model_type == "convolutional":
        model = CNN1D(input_dim=embeds_dim[embeddings_source], num_classes=config.num_labels).to(config.device)
    if model_type == "BiLSTM":
        model = BiLSTM(input_dim=embeds_dim[embeddings_source], num_classes=config.num_labels).to(config.device)

#     optimizer = torch.optim.Adam(model.parameters(), lr = config.lr)
    optimizer = AdamW(model.parameters(), lr=config.lr,weight_decay=1e-6)
#     scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=1)
    scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=800)

    CrossEntropy = torch.nn.CrossEntropyLoss()
    f1_score = MultilabelF1Score(num_labels=config.num_labels).to(config.device)
    n_epochs = config.n_epochs

    print("BEGIN TRAINING...")
    train_loss_history=[]
    val_loss_history=[]
    
    train_f1score_history=[]
    val_f1score_history=[]
    best_score = 0
    best_loss = 1000
    stop_step = 0
    for epoch in range(n_epochs):
        print("EPOCH ", epoch+1)
        ## TRAIN PHASE :
        losses = []
        scores = []
        for embed, targets in tqdm(train_dataloader):
            embed, targets = embed.to(config.device), targets.to(config.device)
            optimizer.zero_grad()
            preds = model(embed)
            loss= CrossEntropy(preds, targets)
            score=f1_score(preds, targets)
            losses.append(loss.item()) 
            scores.append(score.item())
            loss.backward()
            optimizer.step()
        avg_loss = np.mean(losses)
        avg_score = np.mean(scores)

        print("Running Average TRAIN Loss : ", avg_loss)
        print("Running Average TRAIN F1-Score : ", avg_score)
        train_loss_history.append(avg_loss)
        train_f1score_history.append(avg_score)

        ## VALIDATION PHASE : 
        losses = []
        scores = []
        for embed, targets in val_dataloader:
            embed, targets = embed.to(config.device), targets.to(config.device)
            preds = model(embed)
            loss= CrossEntropy(preds, targets)
            score=f1_score(preds, targets)
            losses.append(loss.item())
            scores.append(score.item())
        avg_loss = np.mean(losses)
        avg_score = np.mean(scores)
        print("Running Average VAL Loss : ", avg_loss)
        print("Running Average VAL F1-Score : ", avg_score)
        val_loss_history.append(avg_loss)
        val_f1score_history.append(avg_score)
        
        scheduler.step(avg_loss)
        print("\n")
        if avg_score > best_score and avg_loss < best_loss:
            best_score = avg_score
            best_loss = avg_loss
            stop_step = 0
            torch.save(model.state_dict(), f'{model_aspect}_{embeddings_source}_{config.num_labels}_{model_type}_{best_score}.pth')
        else:
            stop_step += 1
            if stop_step==config.early_stop:
                break
    print("TRAINING FINISHED")
    print("FINAL TRAINING SCORE : ", train_f1score_history[-1])
    print("FINAL VALIDATION SCORE : ", val_f1score_history[-1])
    
    losses_history = {"train" : train_loss_history, "val" : val_loss_history}
    scores_history = {"train" : train_f1score_history, "val" : val_f1score_history}
    del train_dataset, train_set, val_set
    gc.collect()
    return model, losses_history, scores_history

## 7.1. Training Usecase

### BPO_MODEL

In [13]:
# ensem_model_BPO, ensem_losses_BPO, ensem_scores_BPO = train_model(embeddings_source="Ensemble2",model_type="linear",model_aspect="BPO")
# ems2_model_BPO, ems2_losses_BPO, ems2_scores_BPO = train_model(embeddings_source="EMS2",model_type="linear",model_aspect="BPO")
# t5_model_BPO, t5_losses_BPO, t5_scores_BPO = train_model(embeddings_source="T5",model_type="linear",model_aspect="BPO")

In [14]:
# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_losses_BPO["val"], label = "ems2_losses_BPO")
# # plt.plot(t5_losses_BPO["val"], label = "t5_losses_BPO")
# plt.plot(ensem_losses_BPO["val"], label = "ensem_losses_BPO")
# # plt.plot(ProtBERT_losses_BPO["val"], label = "ProtBERT")
# plt.title("Validation Losses for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average Loss")
# plt.legend()
# plt.show()

# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_scores_BPO["val"], label = "ems2_scores_BPO")
# # plt.plot(t5_scores_BPO["val"], label = "t5_scores_BPO")
# plt.plot(ensem_scores_BPO["val"], label = "ensem_scores_BPO")
# # plt.plot(ProtBERT_scores_BPO["val"], label = "ProtBERT")
# plt.title("Validation F1-Scores for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average F1-Score")
# plt.legend()
# plt.show()

### CCO_MODEL

In [15]:
# ensem_model_CCO, ensem_losses_CCO, ensem_scores_CCO = train_model(embeddings_source="Ensemble2",model_type="linear",model_aspect="CCO")
# ems2_model_CCO, ems2_losses_CCO, ems2_scores_CCO = train_model(embeddings_source="EMS2",model_type="linear",model_aspect="CCO")
# t5_model_CCO, t5_losses_CCO, t5_scores_CCO = train_model(embeddings_source="T5",model_type="linear",model_aspect="CCO")

In [16]:
# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_losses_CCO["val"], label = "ems2_losses_CCO")
# # plt.plot(t5_losses_CCO["val"], label = "t5_losses_CCO")
# plt.plot(ensem_losses_CCO["val"], label = "ensem_losses_CCO")
# # plt.plot(protbert_losses["val"], label = "ProtBERT")
# plt.title("Validation Losses for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average Loss")
# plt.legend()
# plt.show()

# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_scores_CCO["val"], label = "ems2_scores_CCO")
# # plt.plot(t5_scores_CCO["val"], label = "t5_scores_CCO")
# plt.plot(ensem_scores_CCO["val"], label = "ensem_scores_CCO")
# # plt.plot(protbert_scores["val"], label = "ProtBERT")
# plt.title("Validation F1-Scores for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average F1-Score")
# plt.legend()
# plt.show()

### MFO_MODEL

In [17]:
# ensem_model_MFO, ensem_losses_MFO, ensem_scores_MFO = train_model(embeddings_source="Ensemble2",model_type="linear",model_aspect="MFO")
# ems2_model_MFO, ems2_losses_MFO, ems2_scores_MFO = train_model(embeddings_source="EMS2",model_type="linear",model_aspect="MFO")
# t5_model_MFO, t5_losses_MFO, t5_scores_MFO = train_model(embeddings_source="T5",model_type="linear",model_aspect="MFO")

In [18]:
# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_losses_MFO["val"], label = "ems2_losses_MFO")
# # plt.plot(t5_losses_MFO["val"], label = "t5_losses_MFO")
# plt.plot(ensem_losses_MFO["val"], label = "ensem_losses_MFO")
# # plt.plot(protbert_losses["val"], label = "ProtBERT")
# plt.title("Validation Losses for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average Loss")
# plt.legend()
# plt.show()

# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_scores_MFO["val"], label = "ems2_scores_MFO")
# # plt.plot(t5_scores_MFO["val"], label = "t5_scores_MFO")
# plt.plot(ensem_scores_MFO["val"], label = "ensem_scores_MFO")
# # plt.plot(protbert_scores["val"], label = "ProtBERT")
# plt.title("Validation F1-Scores for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average F1-Score")
# plt.legend()
# plt.show()

In [19]:
# protbert_model, protbert_losses, protbert_scores = train_model(embeddings_source="ProtBERT",model_type="linear")

In [20]:
# t5_convmodel, t5_convlosses, t5_convscores = train_model(embeddings_source="T5",model_type="convolutional")

In [21]:
# t5_LSTMmodel, t5_LSTMlosses, t5_LSTMscores = train_model(embeddings_source="T5",model_type="BiLSTM")

## 7.3. Train/Val Losses 

In [22]:
# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_losses["val"], label = "EMS2")
# # plt.plot(t5_losses["val"], label = "T5")
# plt.plot(ensem_losses["val"], label = "ensem")
# # plt.plot(protbert_losses["val"], label = "ProtBERT")
# plt.title("Validation Losses for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average Loss")
# plt.legend()
# plt.show()

# plt.figure(figsize = (10, 4))
# # plt.plot(ems2_scores["val"], label = "EMS2")
# # plt.plot(t5_scores["val"], label = "T5")
# plt.plot(ensem_scores["val"], label = "ensem")
# # plt.plot(protbert_scores["val"], label = "ProtBERT")
# plt.title("Validation F1-Scores for # Vector Embeddings")
# plt.xlabel("Epochs")
# plt.ylabel("Average F1-Score")
# plt.legend()
# plt.show()

# 8. Pytorch Lightning module

In [23]:
class Linear_Lightning(pl.LightningModule):
    def __init__(self, input_dim, num_classes, train_size, **hparams):
        super(Linear_Lightning, self).__init__()
        self.linear1 = torch.nn.Linear(input_dim, 1012)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(1012, 712)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(712, num_classes)
        
        train_dataset = ProteinSequenceDataset(datatype="train", embeddings_source = embeddings_source)
        self.train_set, self.val_set = random_split(train_dataset, lengths = [int(len(train_dataset)*train_size), len(train_dataset)-int(len(train_dataset)*train_size)])
        self.loss_fn = torch.nn.CrossEntropyLoss()
        
        self.batch_size = batch_size
        self.lr = lr
        
        self.f1_score = MultilabelF1Score(num_labels=num_classes)
        self.accuracy = MultilabelAccuracy(num_labels=num_classes)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        return x
    
    def training_step(self, batch, batch_idx):
        embed, targets = batch
        preds = self(embed)
        loss = self.loss_fn(preds, targets)
        f1_score = self.f1_score(preds, targets)
        acc_score = self.accuracy(preds, targets)
        
        logs = {"train_loss" : loss, "f1_score" : f1_score, "accuracy_score" : acc_score}
        self.log_dict(
            logs,
            on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"loss":loss, "log" : logs}
    
    def validation_step(self, batch, batch_idx):
        embed, targets = batch
        preds = self(embed)
        loss= self.loss_fn(preds, targets)
        f1_score = self.f1_score(preds, targets)
        acc_score = self.accuracy(preds, targets)
        
        return {"val_loss":loss, "f1_score" : f1_score, "accuracy_score" : acc_score}
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in ouputs]).mean()
        logs = {"val_loss" : avg_loss}
        self.log_dict(
            logs,
            on_step=True, on_epoch=True, prog_bar=True, logger=True
        )
        return {"avg_val_loss" : avg_loss, "log" : logs}
        
    def val_dataloader(self):
        val_dataloader = torch.utils.data.DataLoader(self.val_set, batch_size=config.batch_size, shuffle=False,)
        return val_dataloader


    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        return optimizer
    
    def train_dataloader(self):
        train_dataloader = torch.utils.data.DataLoader(self.train_set, batch_size=self.batch_size, shuffle=False)
        return train_dataloader

## IN PROGRESS
```python
param_grid = {
    "batch_size" : [8, 16, 32, 64],
    "lr" : [1e-4, 1e-3, 1e-2]
}

embeddings_source = "T5"

runs_count=0
for batch_size in param_grid["batch_size"]:
    for lr in param_grid["lr"]:
        runs_count+=1
        print("NEW SESSION RUN (number "+str(runs_count)+")")
        print("batch size : ", batch_size)
        print("learning rate : ", lr)
        
        run_name = "Adam-"+str(batch_size)+"-"+str(lr)
        
        logger = WandbLogger(
            name = run_name,
            project="mlp_model_cafa5",
            job_type="train"
        )

        trainer = Trainer(
            max_epochs=1,
            limit_train_batches=5000,
            logger=logger
        )
        model = Linear_Lightning(
            input_dim=embeds_dim[embeddings_source],
            num_classes=config.num_labels,
            train_size=0.8
        )

        trainer.fit(model)
```

```python
# Define sweep config
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'max': 0.1, 'min': 0.0001}
     }
}

# Initialize sweep by passing in config. 
# (Optional) Provide a name of the project.
sweep_id = wandb.sweep(
  sweep=sweep_configuration, 
  project='my-first-sweep'
  )

# Define training function that takes in hyperparameter 
# values from `wandb.config` and uses them to train a 
# model and return metric
def train_one_epoch(epoch, lr, bs): 
  acc = 0.25 + ((epoch/30) +  (random.random()/10))
  loss = 0.2 + (1 - ((epoch-1)/10 +  random.random()/5))
  return acc, loss

def evaluate_one_epoch(epoch): 
  acc = 0.1 + ((epoch/20) +  (random.random()/10))
  loss = 0.25 + (1 - ((epoch-1)/10 +  random.random()/6))
  return acc, loss

def main():
    run = wandb.init()

    lr  =  wandb.config.lr
    bs = wandb.config.batch_size
    epochs = wandb.config.epochs

    for epoch in np.arange(1, epochs):
      train_acc, train_loss = train_one_epoch(epoch, lr, bs)
      val_acc, val_loss = evaluate_one_epoch(epoch)

      wandb.log({
        'epoch': epoch, 
        'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss
      })

# Start sweep job.
wandb.agent(sweep_id, function=main, count=4)
```

# 9. Make Predictions

In [24]:
def predict(embeddings_source, model, aspect=None):
    test_dataset = ProteinSequenceDataset(datatype="test", embeddings_source = embeddings_source)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
    del test_dataset
    
#     if embeddings_source == "T5":
#         model = t5_model
#     if embeddings_source == "ProtBERT":
#         model = protbert_model
#     if embeddings_source == "EMS2":
#         model = ems2_model
#     if embeddings_source == 'Ensemble':
#         model = ensem_model
    model.eval()
    labels = pd.read_csv(config.train_labels_path, sep = "\t")
    if not aspect:
        print(f"Generate for all aspect")
        top_terms = labels.groupby("term")["EntryID"].count().sort_values(ascending=False)
    else:
        print(f"Generate for {aspect}")
        labels = labels[labels.aspect==f"{aspect}"]
        top_terms = labels.groupby("term")["EntryID"].count().sort_values(ascending=False)
    labels_names = top_terms[:config.num_labels].index.values
    print("GENERATE PREDICTION FOR TEST SET...")

    ids_ = np.empty(shape=(len(test_dataloader)*config.num_labels,), dtype=object)
    go_terms_ = np.empty(shape=(len(test_dataloader)*config.num_labels,), dtype=object)
    confs_ = np.empty(shape=(len(test_dataloader)*config.num_labels,), dtype=np.float32)

    for i, (embed, id) in tqdm(enumerate(test_dataloader),total = len(test_dataloader), position=0, leave=True):
        embed = embed.to(config.device)
        confs_[i*config.num_labels:(i+1)*config.num_labels] = torch.nn.functional.sigmoid(model(embed)).squeeze().detach().cpu().numpy()
        ids_[i*config.num_labels:(i+1)*config.num_labels] = id[0]
        go_terms_[i*config.num_labels:(i+1)*config.num_labels] = labels_names

    submission_df = pd.DataFrame(data={"Id" : ids_, "GO term" : go_terms_, "Confidence" : confs_})
    print("PREDICTIONS DONE")
    gc.collect()
    return submission_df

In [25]:
# del ensem_losses_MFO, ensem_scores_MFO,ensem_losses_BPO, ensem_scores_BPO, ensem_losses_CCO, ensem_scores_CCO
# gc.collect()

In [26]:
ensem_model_BPO = MultiLayerPerceptron(input_dim=2304, num_classes=config.num_labels).to(config.device)
st = torch.load('/kaggle/input/cfca-models/BPO_Ensemble2_500_linear_0.2250145967692545.pth')
ensem_model_BPO.load_state_dict(st)

<All keys matched successfully>

In [27]:
submission_1 = predict("Ensemble2",model=ensem_model_BPO,aspect="BPO")
del ensem_model_BPO,st
gc.collect()
submission_1.to_csv('submission1.tsv', sep='\t', index=False)
del submission_1

Generate for BPO
GENERATE PREDICTION FOR TEST SET...


100%|██████████| 141865/141865 [01:44<00:00, 1360.23it/s]


PREDICTIONS DONE


In [28]:
ensem_model_CCO = MultiLayerPerceptron(input_dim=2304, num_classes=config.num_labels).to(config.device)
st = torch.load('/kaggle/input/cfca-models/CCO_Ensemble2_500_linear_0.14624455499730699.pth')
ensem_model_CCO.load_state_dict(st)


<All keys matched successfully>

In [29]:
submission_2 = predict("Ensemble2",model=ensem_model_CCO,aspect="CCO")
del ensem_model_CCO,st
gc.collect()
submission_2.to_csv('submission2.tsv', sep='\t', index=False)
del submission_2

Generate for CCO
GENERATE PREDICTION FOR TEST SET...


100%|██████████| 141865/141865 [01:44<00:00, 1363.79it/s]


PREDICTIONS DONE


In [30]:
ensem_model_MFO = MultiLayerPerceptron(input_dim=2304, num_classes=config.num_labels).to(config.device)
st = torch.load('/kaggle/input/cfca-models/MFO_Ensemble2_500_linear_0.15245565555749402.pth')
ensem_model_MFO.load_state_dict(st)

<All keys matched successfully>

In [31]:
submission_3 = predict("Ensemble2",model=ensem_model_MFO,aspect="MFO")
del ensem_model_MFO,st
gc.collect()
submission_3.to_csv('submission3.tsv', sep='\t', index=False)
del submission_3

Generate for MFO
GENERATE PREDICTION FOR TEST SET...


100%|██████████| 141865/141865 [01:44<00:00, 1356.60it/s]


PREDICTIONS DONE


In [32]:
# if t5_scores["val"][-1] > ems2_scores["val"][-1]:
#     print('T5')
#     submission_1 = predict("T5")
# else:
#     print('EMS')
#     submission_1 = predict("EMS2")
# submission_3 = predict("ProtBERT")
# submission_1['Confidence'] = 0.5*submission_1['Confidence']+0.4*submission_2['Confidence']+0.1*submission_3['Confidence']
# del submission_2,submission_3
# submission_1.shape,submission_2.shape,submission_3.shape

In [33]:
# submission_1 = pd.concat([submission_1, submission_2, submission_3])

In [34]:
# submission_1.to_csv('submission1.tsv', sep='\t', index=False)